# A. There are different ways to access the files:

## 1.	Map network drive
You can map this new sharepoint site in a similar way as the old one:
Do this only once:
-	Log in to https://wageningenur4.sharepoint.com/sites/LoobosTeamsite with InternetExplorer (the old one, this step does not work with other browsers)
-	Go to settings (gearbox top right) --> Internet Options --> Security tab --> and add https://wageningenur4.sharepoint.com to the list of trusted sites
-	Go to Windows Explorer --> Home (menu bar) --> Easy Access --> Map as Drive --> Select an unused drive letter (e.g. L:\. At least >= D, as A,B,C are reserved for floppy disks and hard disks), copy in to https://wageningenur4.sharepoint.com/sites/LoobosTeamsite into ‘Folder’ and click Finish
-	The Sharepoint should now be available via the drive letter you selected.

Do this each time you want to use the drive letter:
-	Log in in to https://wageningenur4.sharepoint.com/sites/LoobosTeamsite with InternetExplorer (the old one, this step does not work with other browsers)
-	In Windows Explorer, select the drive letter to connect again.

## 2.	Shortcut via OneDrive
-	Open https://wageningenur4.sharepoint.com/sites/LoobosTeamsite in any browser
-	Select ‘Add shortcut to OneDrive’
-	In your OneDrive (either via the browser or WindowsExplorer), the sharepoint is now accessible. In my case it was via C:\Users\molen050\OneDrive – WageningenUR\Documents.
-	You can either use that path as the datapath or assign OneDrive a special drive letter:
-	Hit the windows key and type ‘cmd’ to open a dos command window. Type: subst L: "c:\Users\molen050\OneDrive – WageningenUR\Documents\"
-	The Sharepoint will now be accessible via O:\

## 3.	Download the data to your own drive
-	Simple, reliable, but you will not get updates on the data.



# B.	Reading the data in Python
In the Folder DataShare/PythonScripts/ there are two files:
- a python script Loobos_ReadData.py and 
- a Jupyter Notebook Loobos_ReadData.ipynb. 
They contain the same set of commands as an example of how to read the data and make a figure using the python package pandas.

Let me know if you have remarks or questions.
Michiel

In [ ]:
# Settings
Username   = 'molen050'
years      = range(1997,2021)    #(1997,2021) # Set years to download

In [ ]:
import os
datapath   = "..\\"
print('datapath is set to %s'%datapath)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import cufflinks as cf
from datetime import datetime, timedelta
#import plotly.express as px
#import cufflinks as cf
#cf.go_offline()
#cf.set_config_file(offline=False, world_readable=True)

import sys
sys.path.insert(0, os.path.join(datapath,'PythonScripts'))
from Loobos_Toolbox import Read_LoobosEddFinal, Read_LooStor, Read_LoodatGapfill, Read_Loobos_halfhourly, Read_Loobos_meteo, Read_Loobos_soil, Read_Loobos_profile

In [ ]:
#my own functions
from Loobos_Toolbox_Dendro_Grwater import Read_Dendrometers,notmatch

In [ ]:
# Read files
df_EC           = Read_LoobosEddFinal    (years,datapath)
df_Stor         = Read_LooStor           (years,datapath)
df_Comb         = Read_LoodatGapfill     (years,datapath)
df_NEE          = Read_Loobos_halfhourly (years,datapath)
df_meteo        = Read_Loobos_meteo      (years,datapath)
df_soil         = Read_Loobos_soil       (years,datapath)
df_profile      = Read_Loobos_profile    (years,datapath)


## Import the dinoloket data (location B32F0088)

In [ ]:
#new location put 88
dir = '../Dinoloket_2024-01-20/Grondwaterstanden_Put/B32F0088001_1_edit.csv'
df_GWS=pd.read_csv(dir,sep=",",index_col=False)
df_GWS=df_GWS.drop(['Locatie','Filternummer','Bijzonderheid','Opmerking'], axis=1)
df_GWS['Peildatum']=df_GWS['Peildatum']+" 12:00"
df_GWS['datetime']=pd.to_datetime(df_GWS['Peildatum'], dayfirst=True, format='mixed')
df_GWS.index = df_GWS['datetime']
df_GWS=df_GWS.drop(['Peildatum'], axis=1)
#df_GWS

## Import dendrometer data (old)

#Loobos_dendrometers
dir = '..\Loobos_dendrometers\I5.csv'
df_dendro=pd.read_csv(dir,sep=",",index_col=False)
df_dendro['date2018']="2018" #this is a bit of a goofy way to add te year to groeiseizoen but it should work
df_dendro['date2017']="2017"
df_dendro['date2018']=df_dendro['date2018'].str.cat(df_dendro['groeiseizoen'], join='left', sep='-')
df_dendro['date2017']=df_dendro['date2017'].str.cat(df_dendro['groeiseizoen'], join='left', sep='-')

df_dendro['date2018']=pd.to_datetime(df_dendro['date2018'],dayfirst=False, format='%Y-%m-%d %H:%M') #format='%d-%m-%Y'
df_dendro['date2017']=pd.to_datetime(df_dendro['date2017'],dayfirst=False, format='%Y-%m-%d %H:%M') #format='%d-%m-%Y'

df_dendro2018=pd.concat([df_dendro['date2018'],df_dendro['jr2018']],axis=1,sort=False)
df_dendro2017=pd.concat([df_dendro['date2017'],df_dendro['jr2017']],axis=1,sort=False)

df_dendro2018.index = df_dendro2018['date2018']
df_dendro2017.index = df_dendro2017['date2017']

df_dendro2018['jr2018']

## Import dendrometer data (using function)

In [ ]:
## function to import all dendro data and append it to the other data
df_dendro = Read_Dendrometers(datapath)

In [ ]:
#testing the function (known dry years are 2003 2013 2018. KNMI when looking at droogtemonitor looks at april-sep (months 4-9)
vis_wrapper_drought(time_start="2009-04-01 00:00",
            time_end="2009-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","Bowen_R","dendro_growth_I5","WaterUseEff"],
            yaxis_min_maxes=[[0,0],[0,0],[0,0],[0,0]],
            yaxes_auto=True,
            data=data,
            savefig=False,
            resample="D")
#for resample, "" is no resampling "D" for day.

#df_GWS['Stand (cm t.o.v. MV)']

In [ ]:
#testing the function (known dry years are 2003 2013 2018. KNMI when looking at droogtemonitor looks at april-sep (months 4-9)
vis_wrapper_drought(time_start="2013-04-01 00:00",
            time_end="2013-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","Bowen_R","dendro_growth_I5","WaterUseEff"],
            yaxis_min_maxes=[[0,0],[0,0],[0,0],[0,0]],
            yaxes_auto=True,
            data=data,
            savefig=False,
            resample="D")
#for resample, "" is no resampling "D" for day.

#df_GWS['Stand (cm t.o.v. MV)']

In [ ]:
#testing the function (known dry years are 2003 2013 2018. KNMI when looking at droogtemonitor looks at april-sep (months 4-9)
vis_wrapper_drought(time_start="2014-04-01 00:00",
            time_end="2014-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","Bowen_R","dendro_growth_I5","WaterUseEff"],
            yaxis_min_maxes=[[0,0],[0,0],[0,0],[0,0]],
            yaxes_auto=True,
            data=data,
            savefig=False,
            resample="D")
#for resample, "" is no resampling "D" for day.

#df_GWS['Stand (cm t.o.v. MV)']

In [ ]:
#testing the function (known dry years are 2003 2013 2018. KNMI when looking at droogtemonitor looks at april-sep (months 4-9)
vis_wrapper_drought(time_start="2018-04-01 00:00",
            time_end="2018-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","Bowen_R","dendro_growth_I5","WaterUseEff"],
            #plot_variables=["Stand (cm t.o.v. MV)","Bowen_R_3DSMA","dendro_growth_I5","WaterUseEff"],
            yaxis_min_maxes=[[0,0],[0,0],[0,0],[0,0]],
            yaxes_auto=True,
            data=data,
            savefig=False,
            resample="D")
#for resample, "" is no resampling "D" for day.

#df_GWS['Stand (cm t.o.v. MV)']

In [ ]:
#testing the function (known dry years are 2003 2013 2018. KNMI when looking at droogtemonitor looks at april-sep (months 4-9)
vis_wrapper_drought(time_start="2019-04-01 00:00",
            time_end="2019-09-01 00:00",
            plot_variables=["Stand (cm t.o.v. MV)","Bowen_R","dendro_growth_I5","WaterUseEff"],
            yaxis_min_maxes=[[0,0],[0,0],[0,0],[0,0]],
            yaxes_auto=True,
            data=data,
            savefig=False,
            resample="D")
#for resample, "" is no resampling "D" for day.

#df_GWS['Stand (cm t.o.v. MV)']

In [ ]:
#testing the function for just H and LE
vis_wrapper_heat(
            time_start="19970101 00:00",
            time_end="20201231 23:30",
            plot_variables=["LE","H"],
            yaxis_min_maxes=[[0,0],[0,0]],
            yaxes_auto=True,
            data=data,
            savefig=False,
            resample="M")
#for resample, "" is no resampling "D" for day.

# Old graphs (For reference)

## My plots: Bowen ratio, water use efficiency, light use efficiency

In [ ]:
plt.rcParams["figure.figsize"] = (20,10) #set the default size of graphs to be wide and not very tall

In [ ]:
#SELECT useful data and calculate: bowen ratio is only useful for midday hours, and for when both H and LE are above 50 or 100 W/m2
#step 1: create new dataframe with data of interest
df_bowen = pd.concat([df_Comb['H'],df_Comb['LE'],df_Comb['Hour'],df_Comb['Year']],axis=1,sort=False)

#step 2 select for hours between 9am and 4pm
df_bowen = df_bowen.loc[df_bowen['Hour'] <= 16]
df_bowen = df_bowen.loc[df_bowen['Hour'] >= 9]

#step 3 exclude data where LE or H are below 50
df_bowen = df_bowen.loc[df_bowen['LE'] >= 50]
df_bowen = df_bowen.loc[df_bowen['H'] >= 50]

df_bowen_100 = df_bowen.loc[df_bowen['LE'] >= 100]
df_bowen_100 = df_bowen.loc[df_bowen['H'] >= 100]

#step 4 calulcate bowen ratio
df_bowen['Bowen_R']=df_bowen['H']/df_bowen['LE']
df_bowen_100['Bowen_R']=df_bowen['H']/df_bowen['LE']

#df_bowen

In [ ]:
# plot bowen ratio
fig,ax=plt.subplots()


#ax.plot(df_tmp['Bowen_R'])
ax.plot(df_bowen['Bowen_R'])
#ax.set_ylim(-1000,1000)
plt.title('Bowen ratio (daily average) over time')

#attempting to make axis labels work (neither of these two work)
#plt.xlablel('Bowen ratio [-]')
#ax.set_xticks(l)

#attempting to make it so there are more x ticks than just seven (doesnt work, will just print all of them on the far left)
#relevant functions are: set_xticks() and set_xticklabels()


plt.show()


In [ ]:
#resampling the dataset so it's a bit more intelligible in a plot
df_bowen_daymean50=df_bowen.resample('D').mean()
df_bowen_daymean100=df_bowen_100.resample('D').mean()
#df_bowen_daymax=df_bowen.resample('D').max()

In [ ]:
# plot bowen ratio
fig,ax=plt.subplots()
#from datetime import datetime, timedelta

#ax.plot(df_tmp['Bowen_R'])
ax.plot(df_bowen_daymean100['Bowen_R'])

#plt.grid(color='b', linestyle='-', linewidth=1, which='Major', markevery=0.5)
#start, end = ax.get_xlim()
#t = np.arange(start, end, timedelta(years=1)).astype(datetime)
#ax.xaxis.set_ticks(np.arange(start, end, timedelta(days=356)).astype(datetime)) #doesnt seem to work
ax.xaxis.set_ticks([datetime(1997,1,1),
                    datetime(1998,1,1),
                    datetime(1999,1,1),
                    datetime(2000,1,1),
                    datetime(2001,1,1),
                    datetime(2002,1,1),
                    datetime(2003,1,1),
                    datetime(2004,1,1),
                    datetime(2005,1,1),
                    datetime(2006,1,1),
                    datetime(2007,1,1),
                    datetime(2008,1,1),
                    datetime(2009,1,1),
                    datetime(2010,1,1),
                    datetime(2011,1,1),
                    datetime(2012,1,1),
                    datetime(2013,1,1),
                    datetime(2014,1,1),
                    datetime(2015,1,1),
                    datetime(2016,1,1),
                    datetime(2017,1,1),
                    datetime(2018,1,1),
                    datetime(2019,1,1),
                    datetime(2020,1,1),
                    datetime(2021,1,1)])
#ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%0.1f'))
#ax.xaxis.set_major_formatter(ticker.Year)
plt.xticks(rotation=45)

#ax.set_ylim(-1000,1000)
plt.title('Bowen ratio (daily average) over time, for hours 09-16 and both LE and H > 100W/m2')

plt.ylabel('Bowen ratio [-]')
plt.xlabel('Time')
#attempting to make axis labels work (neither of these two work)
#plt.xlablel('Bowen ratio [-]')
#ax.set_xticks(l)

#attempting to make it so there are more x ticks than just seven (doesnt work, will just print all of them on the far left)
#relevant functions are: set_xticks() and set_xticklabels()

#x = [0,5,9,10,15]
#y = [0,1,2,3,4]
#fig, ax = plt.subplots()
#ax.plot(x,y)
#start, end = ax.get_xlim()
#ax.xaxis.set_ticks(np.arange(start, end, 0.712123))
#ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%0.1f'))
#plt.show()


plt.show()

### Water Use Efficiency

In [ ]:
#SELECT useful data and calculate: water use efficiency
#step 1: create new dataframe with data of interest
df_WUE = pd.concat([df_Comb['GPP_f'],df_Comb['LE'],df_Comb['Hour'],df_Comb['Year']],axis=1,sort=False)

#step 2 select for hours between 9am and 4pm
df_WUE = df_WUE.loc[df_WUE['Hour'] <= 16]
df_WUE = df_WUE.loc[df_WUE['Hour'] >= 9]

#step 3 exclude data where LE or H are below 50
df_WUE = df_WUE.loc[df_WUE['LE'] >= 100]
#df_bowen = df_bowen.loc[df_bowen['H'] >= 50]

#df_bowen_100 = df_bowen.loc[df_bowen['LE'] >= 100]
#df_bowen_100 = df_bowen.loc[df_bowen['H'] >= 100]



#step 4 calulcate bowen ratio
df_WUE['WaterUseEff']=df_WUE['GPP_f']/df_WUE['LE']

df_WUE_daymean=df_WUE.resample('D').mean()

#df_bowen

In [ ]:
# plot water use efficiency
fig,ax=plt.subplots()

#ax.plot(df_tmp['Bowen_R'])
ax.plot(df_WUE_daymean['WaterUseEff'])
ax.set_ylim(-0.1,0.3)

ax.xaxis.set_ticks([datetime(1997,1,1),
                    datetime(1998,1,1),
                    datetime(1999,1,1),
                    datetime(2000,1,1),
                    datetime(2001,1,1),
                    datetime(2002,1,1),
                    datetime(2003,1,1),
                    datetime(2004,1,1),
                    datetime(2005,1,1),
                    datetime(2006,1,1),
                    datetime(2007,1,1),
                    datetime(2008,1,1),
                    datetime(2009,1,1),
                    datetime(2010,1,1),
                    datetime(2011,1,1),
                    datetime(2012,1,1),
                    datetime(2013,1,1),
                    datetime(2014,1,1),
                    datetime(2015,1,1),
                    datetime(2016,1,1),
                    datetime(2017,1,1),
                    datetime(2018,1,1),
                    datetime(2019,1,1),
                    datetime(2020,1,1),
                    datetime(2021,1,1)])

plt.xticks(rotation=45)

plt.title('Water use efficiency (daily average) over time, for hours 09-15 and LE > 100Wm/2')

plt.show()

## Plotting Bowen ratio and WUE combined

In [ ]:
#example adapted from https://matplotlib.org/stable/gallery/subplots_axes_and_figures/two_scales.html
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('time')
ax1.set_ylabel('Bowen ratio', color=color)
ax1.plot(df_bowen_daymean100['Bowen_R'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax1.xaxis.set_ticks([datetime(1997,1,1),
                    datetime(1998,1,1),
                    datetime(1999,1,1),
                    datetime(2000,1,1),
                    datetime(2001,1,1),
                    datetime(2002,1,1),
                    datetime(2003,1,1),
                    datetime(2004,1,1),
                    datetime(2005,1,1),
                    datetime(2006,1,1),
                    datetime(2007,1,1),
                    datetime(2008,1,1),
                    datetime(2009,1,1),
                    datetime(2010,1,1),
                    datetime(2011,1,1),
                    datetime(2012,1,1),
                    datetime(2013,1,1),
                    datetime(2014,1,1),
                    datetime(2015,1,1),
                    datetime(2016,1,1),
                    datetime(2017,1,1),
                    datetime(2018,1,1),
                    datetime(2019,1,1),
                    datetime(2020,1,1),
                    datetime(2021,1,1)])

plt.xticks(rotation=45)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Water Use Efficiency', color=color)  # we already handled the x-label with ax1
ax2.plot(df_WUE_daymean['WaterUseEff'], color=color)
ax2.set_ylim(-0.1,0.3)
ax2.tick_params(axis='y', labelcolor=color)

fig.suptitle('Bowen ratio and Water use efficiency')
fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()


In [ ]:
#do it again but split the datasets into two datasets of 12 years each.
df_bowen_97_09=df_bowen_daymean100.loc[df_bowen_daymean100['Year']>=1997].loc[df_bowen_daymean100['Year']<=2009] 
df_bowen_09_21=df_bowen_daymean100.loc[df_bowen_daymean100['Year']>=2010].loc[df_bowen_daymean100['Year']<=2021]

df_WUE_97_09=df_WUE_daymean.loc[df_WUE_daymean['Year']>=1997].loc[df_WUE_daymean['Year']<=2009]
df_WUE_09_21=df_WUE_daymean.loc[df_WUE_daymean['Year']>=2010].loc[df_WUE_daymean['Year']<=2021]

In [ ]:
#example adapted from https://matplotlib.org/stable/gallery/subplots_axes_and_figures/two_scales.html
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('time')
ax1.set_ylabel('Bowen ratio', color=color)
ax1.plot(df_bowen_97_09['Bowen_R'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax1.xaxis.set_ticks([datetime(1997,1,1),
                    datetime(1998,1,1),
                    datetime(1999,1,1),
                    datetime(2000,1,1),
                    datetime(2001,1,1),
                    datetime(2002,1,1),
                    datetime(2003,1,1),
                    datetime(2004,1,1),
                    datetime(2005,1,1),
                    datetime(2006,1,1),
                    datetime(2007,1,1),
                    datetime(2008,1,1),
                    datetime(2009,1,1),
                    datetime(2010,1,1)])

plt.xticks(rotation=45)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Water Use Efficiency', color=color)  # we already handled the x-label with ax1
ax2.plot(df_WUE_97_09['WaterUseEff'], color=color)
ax2.set_ylim(-0.1,0.3)
ax2.tick_params(axis='y', labelcolor=color)

fig.suptitle('Bowen ratio and Water use efficiency (1997-2009)')
fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

In [ ]:
#example adapted from https://matplotlib.org/stable/gallery/subplots_axes_and_figures/two_scales.html
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('time')
ax1.set_ylabel('Bowen ratio', color=color)
ax1.plot(df_bowen_09_21['Bowen_R'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax1.xaxis.set_ticks([datetime(2010,1,1),
                    datetime(2011,1,1),
                    datetime(2012,1,1),
                    datetime(2013,1,1),
                    datetime(2014,1,1),
                    datetime(2015,1,1),
                    datetime(2016,1,1),
                    datetime(2017,1,1),
                    datetime(2018,1,1),
                    datetime(2019,1,1),
                    datetime(2020,1,1),
                    datetime(2021,1,1)])

plt.xticks(rotation=45)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Water Use Efficiency', color=color)  # we already handled the x-label with ax1
ax2.plot(df_WUE_09_21['WaterUseEff'], color=color)
ax2.set_ylim(-0.1,0.3)
ax2.tick_params(axis='y', labelcolor=color)

fig.suptitle('Bowen ratio and Water use efficiency (2010-2021)')
fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

## Light use Efficiency

In [ ]:
#SELECT useful data
#step 1: create new dataframe with data of interest
df_LUE = pd.concat([df_meteo['S(i)'],df_Comb['GPP_f'],df_Comb['Hour'],df_Comb['Year']],axis=1,sort=False)

#step 2 select for daylight hours by using S(i)
df_LUE = df_LUE.loc[df_LUE['S(i)'] >= 100]

#too much data for a scatterplot, split into 4 periods of 6 years (2021-1997 = 24. 24/4 = 6)
#1997-2003 2004-2009 2010-2015 2016-2021

df_LUE_1 = df_LUE.loc[df_LUE['Year'] >=1997].loc[df_LUE['Year'] <=2003 ]
df_LUE_2 = df_LUE.loc[df_LUE['Year'] >=2004].loc[df_LUE['Year'] <=2009 ]
df_LUE_3 = df_LUE.loc[df_LUE['Year'] >=2010].loc[df_LUE['Year'] <=2015 ]
df_LUE_4 = df_LUE.loc[df_LUE['Year'] >=2016].loc[df_LUE['Year'] <=2021 ]

In [ ]:
# plot light use efficiency
fig,ax=plt.subplots()

#df_LUE = df_LUE.loc[df_LUE['Year'] == 1998.0]

#df_LUE=df_LUE.resample('D').mean()

ax.scatter(x=df_LUE['S(i)'],y=df_LUE['GPP_f'])
ax.set_ylim(-20,50)
plt.ylabel('GPP (gapfilled) [gC/m2year]')
plt.xlabel('S(i) [W/m2]')
plt.title('Light use efficiency over time')



plt.show()

In [ ]:
# plot light use efficiency in 4 periods of 6 years
fig,axes=plt.subplots(nrows=2,ncols=2)
fig.suptitle('Light use efficiency (GPP/Rs) (halfhourly, for Rs>100W/m)')

#df_LUE = df_LUE.loc[df_LUE['Year'] == 1998.0]

#df_LUE=df_LUE.resample('D').mean()

a=axes[0,0].scatter(x=df_LUE_1['S(i)'],y=df_LUE_1['GPP_f'],c=df_LUE_1.Year,cmap='viridis',s=5)
axes[0,0].set_ylim(-20,50)
axes[0,0].set_title('1997-2003')
b=axes[0,1].scatter(x=df_LUE_2['S(i)'],y=df_LUE_2['GPP_f'],c=df_LUE_2.Year,cmap='viridis',s=5)
axes[0,1].set_ylim(-20,50)
axes[0,1].set_title('2004-2009')
c=axes[1,0].scatter(x=df_LUE_3['S(i)'],y=df_LUE_3['GPP_f'],c=df_LUE_3.Year,cmap='viridis',s=5)
axes[1,0].set_ylim(-20,50)
axes[1,0].set_title('2010-2015')
d=axes[1,1].scatter(x=df_LUE_4['S(i)'],y=df_LUE_4['GPP_f'],c=df_LUE_4.Year,cmap='viridis',s=5)
axes[1,1].set_ylim(-20,50)
axes[1,1].set_title('2016-2021')

for axi in axes:
    for ax in axi:
        ax.set_ylabel('GPP (gapfilled) [gC/m2year]')
        ax.set_xlabel('S(i) [W/m2]')

#import matplotlib.dates as mdates
#myFmt = mdates.DateFormatter('%d')
#ax.xaxis.set_major_formatter(myFmt)

plt.colorbar(a,ax=axes[0,0]) # seems to work
plt.colorbar(b,ax=axes[0,1])
plt.colorbar(c,ax=axes[1,0])
plt.colorbar(d,ax=axes[1,1])


plt.show()

In [ ]:
#Plot all 3 combined
df_plot_multi = pd.concat([df_plot['LightUseEff'],df_Comb_res['WaterUseEff'],df_Comb_res['Bowen_R']],axis=1,sort=False)
df_plot_multi.columns = ['LightUseEff','WaterUseEff','Bowen_R']
f1 = df_plot_multi.iplot(asFigure=True, layout=dict(yaxis=dict(title=''), xaxis=dict(title='time')), width=2)

#f1.ylim(-2000,2000) #(doesnt work)
f1.show()

In [ ]:
# plot bowen ratio
#df_Comb['WaterUseEff']=df_Comb['GPP_f']/df_Comb['LE']
#df_Comb.groupby(df_Comb['Hour'])['WaterUseEff'].mean().iplot(asFigure=True,layout=dict(yaxis=dict(title='water use efficiency'), xaxis=dict(title='time')), width=2)


In [ ]:
print(df_Comb.index.min())
print(df_meteo.index.min())
print(df_Comb.index.max())
print(df_meteo.index.max())

### Bonus: GPP & LE

In [ ]:
#SELECT useful data and calculate: water use efficiency
#step 1: create new dataframe with data of interest
df_WUE = pd.concat([df_Comb['GPP_f'],df_Comb['LE'],df_Comb['Hour']],axis=1,sort=False)

#step 2 select for hours between 9am and 4pm
#df_WUE = df_WUE.loc[df_WUE['Hour'] <= 16]
#df_WUE = df_WUE.loc[df_WUE['Hour'] >= 9]

#step 3 exclude data where LE or H are below 50
#df_WUE = df_WUE.loc[df_WUE['LE'] >= 50]
#df_bowen = df_bowen.loc[df_bowen['H'] >= 50]

#df_bowen_100 = df_bowen.loc[df_bowen['LE'] >= 100]
#df_bowen_100 = df_bowen.loc[df_bowen['H'] >= 100]



#step 4 calulcate bowen ratio
df_WUE['WaterUseEff']=df_WUE['GPP_f']/df_WUE['LE']

#df_bowen

In [ ]:
# plot water use efficiency
fig,ax=plt.subplots()

ax.plot(df_WUE['GPP_f'])
ax.set_ylim(-50,80)

plt.title('GPP_f (halfhourly)')

plt.show()

In [ ]:
#SELECT useful data and calculate: water use efficiency
#step 1: create new dataframe with data of interest
df_WUE = pd.concat([df_Comb['GPP_f'],df_Comb['LE'],df_Comb['Hour']],axis=1,sort=False)

#step 2 select for hours between 9am and 4pm
df_WUE = df_WUE.loc[df_WUE['Hour'] <= 16]
df_WUE = df_WUE.loc[df_WUE['Hour'] >= 9]

#step 3 exclude data where LE or H are below 50
#df_WUE = df_WUE.loc[df_WUE['LE'] >= 50]
#df_bowen = df_bowen.loc[df_bowen['H'] >= 50]

#df_bowen_100 = df_bowen.loc[df_bowen['LE'] >= 100]
#df_bowen_100 = df_bowen.loc[df_bowen['H'] >= 100]



#step 4 calulcate bowen ratio
df_WUE['WaterUseEff']=df_WUE['GPP_f']/df_WUE['LE']

df_WUE_daymean=df_WUE.resample('D').mean()


#df_bowen

In [ ]:
# plot water use efficiency
fig,ax=plt.subplots()

ax.plot(df_WUE_daymean['GPP_f'])
ax.set_ylim(-50,80)

plt.title('GPP_f (daily mean)')

plt.show()

In [ ]:
# plot water use efficiency
fig,ax=plt.subplots()

ax.plot(df_WUE_daymean['LE'])
#ax.set_ylim(-50,80)

plt.title('LE (daily mean)')

plt.show()